# Unit 8: Image Analysis Using `skimage`
----------------------------------------

The result of some experiments are captured in a image files, which we can process to create a quantitative result. Some examples of images that we might need to process:

- Counting particle sizes in a photomicrograph
- Measuring % corrosion based on color shift

Beyond these, there may be instances where it would be helpful to use image processing techniques to pre-process files for machine learning. This might include exposure compensation and cropping the image. 

Two important packages for image processing are:

1. [`scipy.ndimage`](https://docs.scipy.org/doc/scipy/reference/ndimage.html)
2. [*scikit-image* (`skimage`)](https://scikit-image.org/docs/stable/auto_examples/)

**After completing this unit, you should be able to:**

- Load an image file into memory
- Crop and display the image file
- Create a histogram of brightness and/or the individual color channels
- Apply a threshold to convert an image to black/white
- Count objects in an image


## 8.1. Loading and viewing images

### 8.1.1. Creating an `ndarray` from an image

There are many file formats that have been developed to efficiently store images, such as *.png*, *.jpg* and *.gif*. How exactly the image is stored and compressed and stored on the disk is not important to us, as long as we can extract the data out and work with the individual pixels. The package [*scikit-image* (`skimage`)](https://scikit-image.org/) provides this functionality. 

First, we import the subpackage `skimage.io`. Then we can call the `io.imread()` function to load the file and convert it to an `np.ndarray` object. Once the file is loaded, it makes no difference what the original file type was. The returned `np.ndarray` may be in one of 3 forms:

1. $M \times N$ (2d array): Used for grayscale images.
2. $M \times N \times 3$ (3d array): Used for RGB (red, green, blue) channels in a color image.
3. $M \times N \times 4$ (3d array): Used for RGBA (red, green, blue, alpha) channels in a color image. The alpha channel provides the opacity value for the pixel. An alpha value of 0 means that the pixel is fully transparent.

When the `as_gray=True` parameter is set, a color image will be converted to grayscale. In this case the values in the `np.ndarray` will be floating point values from [0.0, 1.0], where 0 is black and 1 is white.

Once the image is loaded, we can treat it as any other array. For instance, we can access the `ndarray.shape` property to see which of the above formats the image has taken. We see that this image has loaded as an RGB color image, with a height (rows) of 800 pixels and a width (columns) of 1200 pixels. 

The `matplotlib` package provides a function specific to displaying image arrays called `axis.imshow()`. Notice that the $\left( 0,0 \right)$ point is in the top left of the image. If you choose, you can uncomment the `axis.set_axis_off()` function to remove the frame and ticks from the image display.

In [ ]:
from skimage import io
import matplotlib.pyplot as plt

# load the file into a np.ndarray
img = io.imread('../../data/prairie_chicken.jpg')
print(f'Array shape: {img.shape}')

# display the image using matplotlib
fig, ax = plt.subplots()
ax.imshow(img)
# ax.set_axis_off()


### 8.1.2. Slicing (cropping) the image

With the image in an `np.ndarray`, we can take advantage of the same slicing syntax that we have used in previous units. In photo editing terms, we can interpret this as cropping the image. Because this is an color image, we need to select all color channels. Edit the code below to select different sections of the image.

In [ ]:
fig, ax = plt.subplots()

ax.imshow(img[200:600, 250:550, :])
ax.set_axis_off()

### 8.1.3. Viewing the image histogram

The histogram of the color channels and the overall image are commonly used in photography to evaluate the exposure of a photograph. Because the image data is stored in an `np.ndarray`, we can generate histograms using the same methods that we have used in previous units. In the code below, we slice the 3d array to generate the histogram of each color channel separately. Additionally, you'll notice that the function 

`img[:, :, 0]` selects all rows / all columns of channel 0 (red), which returns a 2d array

`img[:, :, 0].ravel()` takes the 2d array returned by the slicing operation and reshapes it into a 1d array for the histogram. If we don't do this, then `matplotlib` will try to generate separate histograms for each row of the image.

In [ ]:
plt.style.use('ggplot')

fig, ax = plt.subplots()

# create a histogram for each color channel in the RGB image
# the alpha parameter is used to produce transparent overlapping histograms
ax.hist(img[:, :, 0].ravel(), density=True, bins=20, 
        color='red', alpha=0.4, label='Red Channel')
ax.hist(img[:, :, 1].ravel(), density=True, bins=20, 
        color='blue', alpha=0.4, label='Blue Channel')
ax.hist(img[:, :, 2].ravel(), density=True, bins=20, 
        color='green' ,alpha=0.4, label='Green Channel')

ax.legend()
ax.set_xlabel(r'$\longleftarrow$ darker / lighter $\longrightarrow$')

Alternatively, we can set up a faceted plot to view the color channel and histogram together. By defining a function, as shown below, we can reduce the amount of redundant code that needs to be written. 

In [ ]:
def plot_channel(img_arr, channel_index, color):

    # display the image color channel in the top row
    ax[0, channel_index].imshow(img_arr[:, :, channel_index],
                                cmap=f'{color}s')
    ax[0, channel_index].set_axis_off()

    # display the color channel histogram in the lower row
    ax[1, channel_index].hist(img_arr[:, :, channel_index].ravel(), 
                              color=color, density=True)
    ax[1, channel_index].set_xlabel(f'{color} channel')


fig, ax = plt.subplots(nrows=2, ncols=3, sharex='row', sharey='row', dpi=150)

# call custom function to plot the individual channel
plot_channel(img, 0, 'Red')
plot_channel(img, 1, 'Green')
plot_channel(img, 2, 'Blue')


## 8.2. Counting particles in an image

There are many instances where counting (and measuring) discrete particles in an image can be useful. For instance, maybe we want to estimate a particle size distribution based on a calibrated photomicrograph. Or, we can use this technique to count and measure defects in an extruded polymer film. The basic steps that we will apply to identify particles in will be the same for both of these problems.

*scikit-image* provides a set of example images in the [`skimage.data`](https://scikit-image.org/docs/stable/auto_examples/index.html#data) subpackage that we can use to test algorithms. For this example, we'll use a deep-space image from the Hubble telescope. This image can be loaded by calling the function `data.hubble_deep_field()`, as shown in the example below. Follow the link to find other sample images that you can experiment with.

### 8.2.1. Converting the color image to grayscale

For this example, we're not concerned what color the objects are - only that they are much brighter than their surroundings. For this problem it is convenient to convert the image to grayscale, versus dealing with each color channel. The `data.hubble_deep_field()` function returns an color image with a shape of $M \times N \times 3$. If you look at the `ndarray.max()` of the original image, you'll see that the values in the color channel range from $\left[0,255\right]$, meaning that this image is 8-bit per channel. The `skimage.color` package provides a function `color.rgb2gray()` which will take the original RGB image and convert it to a grayscale array of shape $M \times N$. The grayscale array is stored as floating point values $\left[0,1\right]$, with $0=black$ and $1=white$.

Note the use of the `cmap` parameter in the `axis.imshow()` function that displays the grayscale image. The series of numbers $\left[0,1\right]$ stored in this array can be mapped to any color scale that we choose. Here, we have selected the colormap `gray`, which maps the minimum value of the array to black and the maximum value of the array to white. The *matplotlib* package contains many alternate colormaps that may be useful for different applications, and the web documentation has a [colormap reference](https://matplotlib.org/stable/tutorials/colors/colormaps.html) that may be useful to you. In the code below, experiment with some of the different colormaps from the documentation by changing the value of the `cmap` argument. The colormap names are case-sensitive, so match the value exactly. Additionally, you can add `_r` at the end of the colormap name to reverse the mapping.

In [ ]:
from skimage import data, color

# load the hubble image that is packaged with scikit-image
hubble_img = data.hubble_deep_field()
hubble_gray_img = color.rgb2gray(hubble_img)


fig, ax = plt.subplots(ncols=2, dpi=150)
ax[0].imshow(hubble_img)
ax[0].set_axis_off()
ax[0].set_title(f'Original Image: shape={hubble_img.shape}', fontsize='x-small')

ax[1].imshow(hubble_gray_img, cmap='gray')
ax[1].set_axis_off()
ax[1].set_title(f'Grayscale Image: shape={hubble_gray_img.shape}', fontsize='x-small')


### 8.2.2. Applying a threshold to convert to a (binary) black/white image

Before we can count the objects in this image, we need to convert the image to a true black and white (binary) image. Instead of shades of gray, represented by a real number from $\left[0,1\right]$, this will represent the pixels as either $0$ (black) or $1$ (white) only. This step will explicitly define where the edge of the object is. 

We can visualize the grayscale brightness as a histogram, as we have previously done. Observe that most of the image has an brightness of less than 0.1, which should not surprise us (since the image is mostly black).

In [ ]:
# view the brightness histogram for the grayscale Hubble image
fig, ax = plt.subplots()
ax.hist(hubble_gray_img.ravel(), bins=20, density=True)

ax.set_xlabel(r'$\longleftarrow$ (darker) Grayscale Value (lighter) $\longrightarrow$')

One approach to set the pixels to black and white is to arbitrarily select a threshold value. Pixels that are brighter than this value will be considered white and pixels darker than this value will be considered black. Experiment on your own with changing this value and see how the image changes.

In [ ]:
# choose a threshold brightness, and create a True (1) / False (0) matrix
ARBITRARY_THRESHOLD = 0.5
hubble_bw_img = hubble_gray_img > ARBITRARY_THRESHOLD

# create a 2x2 figure to plot images on the top row and histograms below
fig, ax = plt.subplots(nrows=2, ncols=2, dpi=150, sharey='row')

# plot original grayscale image and histogram on the left
ax[0, 0].imshow(hubble_gray_img, cmap='gray')
ax[0, 0].set_axis_off()
ax[0, 0].set_title(f'Grayscale Image', fontsize='x-small')

ax[1, 0].hist(hubble_gray_img.ravel(), bins=20, density=True)
ax[1, 0].set_xlabel(r'$\longleftarrow$ (darker) Grayscale Value (lighter) $\longrightarrow$', fontsize='x-small')

# plot the black/white image and histogram on the right
ax[0, 1].imshow(hubble_bw_img, cmap='gray')
ax[0, 1].set_axis_off()
ax[0, 1].set_title(f'Black/White (Binary) Image', fontsize='x-small')

# to generate the histogram we need to convert the boolean (True/False) ndarray to a number using .astype(float)
ax[1, 1].hist(hubble_bw_img.ravel().astype(float), bins=20, density=True)
ax[1, 1].set_xlabel(r'$\longleftarrow$ (darker) Grayscale Value (lighter) $\longrightarrow$', fontsize='x-small')

In the previous example, we chose the threshold value to use and manually coded this into our analysis. There are other known methods for generating this threshold based on the image itself, which are provided for your use in the [`skimage.filters`](https://scikit-image.org/docs/stable/api/skimage.filters.html) subpackage. There is a good [overview on thresholding](https://scikit-image.org/docs/stable/auto_examples/applications/plot_thresholding.html) provided on the *scikit-image* site to help you choose a method. Keep in mind that the best method for one type of image may not be ideal for another.

As an example, we will automatically select a threshold value using [Otsu's Method](https://scikit-image.org/docs/stable/api/skimage.filters.html#skimage.filters.threshold_otsu). Before calling the `threshold_otsu()` function, it needs to be imported from the `skimage.filters` package. The grayscale image is passed to the `threshold_otsu` function, which returns the value for the threshold. Then, we can use this threshold value as before to determine whether a pixel should be regarded as black or white.

In [ ]:
# import the selected threshold function
from skimage.filters import threshold_otsu

# calculate the threshold value and apply it to the grayscale image
hubble_gray_threshold = threshold_otsu(hubble_gray_img)
hubble_bw_img = hubble_gray_img > hubble_gray_threshold

# plot the histogram for the grayscale image
fig, ax = plt.subplots()
ax.hist(hubble_gray_img.ravel(), bins=20, density=True)

ax.set_ylim((0, 10))
ax.set_xlabel(r'$\longleftarrow$ (darker) Grayscale Value (lighter) $\longrightarrow$')

# add a dashed line indicating the calculated threshold
ax.axvline(hubble_gray_threshold, ls='--')
ax.text(hubble_gray_threshold+0.02, 9, r'white $\longrightarrow$', ha='left')
ax.text(hubble_gray_threshold-0.02, 9, r'$\longleftarrow$ black', ha='right')


As before, we can view the grayscale and black/white images side-by-side for comparison. To illustrate how different colormaps can change how the image is visualized, I'll use the reversed grayscale colormap, `gray_r`, for this example.

In [ ]:
fig, ax = plt.subplots(ncols=2, dpi=150)

# plot the original grayscale image
ax[0].imshow(hubble_gray_img, cmap='gray_r')
ax[0].set_axis_off()
ax[0].set_title(f'Grayscale Image', fontsize='x-small')

# plot the black/white image generated by Otsu's method
ax[1].imshow(hubble_bw_img, cmap='gray_r')
ax[1].set_axis_off()
ax[1].set_title('After Otsu\'s Method Threshold', fontsize='x-small')

### 8.2.3. Counting objects in the image

In [ ]:
from scipy import ndimage
from skimage import measure
import pandas as pd

labels, no_objects = ndimage.label(hubble_bw_img)

props_table = measure.regionprops_table(labels, properties=['centroid', 
                                                            'area',
                                                            'bbox'])

props_df = pd.DataFrame(props_table)
props_df.head()

We can look at the labels array to see exactly what the `ndimage.label()` function is doing. The array is of the same shape as the black/white image. In the code below, we identify the corner points for the 10th object that was identified by the `ndimage.label()` function. On the left, we plot a annotated heatmap of the black/white image. Notice that the object has an area of 5 pixels, and these pixels are labelled as $1$, because they are white in the image. On the right, there is another annotated heatmap from the same area in the `label` array. This object is labelled as `11`. It is not counted as `10`, because no object can be labelled as `0` (it wouldn't be distinguishable from the background). There is a numbered region for each object identified in the image. 

In [ ]:
import seaborn as sns

row_min = int(props_df.loc[10]['bbox-0'] - 2)
row_max = int(props_df.loc[10]['bbox-2'] + 2)
col_min = int(props_df.loc[10]['bbox-1'] - 2)
col_max = int(props_df.loc[10]['bbox-3'] + 2)

fig, ax = plt.subplots(ncols=2, figsize=(6,3))

# plot a heatmap with the values from the black/white image
sns.heatmap(hubble_bw_img[row_min:row_max, col_min:col_max], annot=True, fmt="d", linewidths=.5, ax=ax[0])
ax[0].set_axis_off()
ax[0].set_title('Black/White Image (1=white)', fontsize='small')

# plot a heatmap with the corresponding values from the labels array
sns.heatmap(labels[row_min:row_max, col_min:col_max], annot=True, fmt="d", linewidths=.5, ax=ax[1])
ax[1].set_axis_off()
ax[1].set_title('Label Array', fontsize='small')

The bounding box, `bbox`, contains the minimum and maximum for the rows and columns.

- `bbox-0`: minimum row touched by the object
- `bbox-1`: minimum column touched by the object
- `bbox-2`: maximum row touched by the object
- `bbox-3`: maximum column touched by the object

In [ ]:
from matplotlib.patches import Rectangle

fig, ax = plt.subplots(dpi=150)

ax.imshow(hubble_bw_img, cmap='gray_r')

for index, row_data in props_df.iterrows():

    corner = (row_data['bbox-1']-1, row_data['bbox-0']-1)
    height = row_data['bbox-2'] - row_data['bbox-0'] + 1
    width = row_data['bbox-3'] - row_data['bbox-1'] + 1

    ax.patches.append(Rectangle(corner, width, height, 
                                color='red', alpha=0.3))

ax.set_axis_off()
ax.set_title(f'Black/White (Binary) Image', fontsize='x-small')

In [ ]:
fig, ax = plt.subplots()
ax.hist(props_df['area'], bins=20, density=True)
ax.set_xlabel('Area of Object (pixels)')

In [ ]:
from skimage.morphology import remove_small_objects

MIN_PIXEL_AREA = 50
hubble_cleaned_img = remove_small_objects(hubble_bw_img, MIN_PIXEL_AREA)

fig, ax = plt.subplots(ncols=2, dpi=150)

ax[0].imshow(hubble_bw_img, cmap='gray_r')
ax[0].set_title('Original black/white image', fontsize='x-small')
ax[0].set_axis_off()

ax[1].imshow(hubble_cleaned_img, cmap='gray_r')
ax[1].set_title(f'Objects < {MIN_PIXEL_AREA} pixels removed', fontsize='x-small')
ax[1].set_axis_off()


In [ ]:
labels_cleaned, no_objects_cleaned = ndimage.label(hubble_cleaned_img)

props_table_cleaned = measure.regionprops_table(labels_cleaned, 
                                                properties=['centroid', 
                                                            'area',
                                                            'bbox'])

props_df_cleaned = pd.DataFrame(props_table_cleaned)

fig, ax = plt.subplots()

ax.hist(props_df_cleaned['area'], bins=20, density=True)

--------------
## Next Steps:

1. Complete the [Unit 8 Problems](./unit08-solutions.ipynb) to test your understanding